# Shortest path with time constraints

![title](img/ej9.png)

### If a person had to travel between s and t in less than 9 hours (T). What’s the shortest path? Try to solve the problem with a simple LP model.
### What if the maximum available time that this person has drops to 8 hours? What’s the new shortest path? Understand the LP model outputs.
### What’s the first solution that comes to your mind in order to solve point 2 issues? Is it feasible in reality?

# Import modules and libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

from mis_utils import *

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.latex.repr = True

## Set connections and get matrices

In [2]:
# Definiciones de constantes

node_names = np.array(('s', '2','3', '4', '5', 't'))

NN_dist = np.zeros((node_names.shape[0], node_names.shape[0]))
NN_time = np.zeros((node_names.shape[0], node_names.shape[0]))

# b = np.zeros((node_names.shape))
beq = np.zeros((node_names.shape))
beq[0]=1
beq[-1]=-1
T1 = 9
T2 = 8

In [3]:
connect_nodes(NN_dist, node_names, 's', '2', 2)
connect_nodes(NN_time, node_names, 's', '2', 3)

connect_nodes(NN_dist, node_names, '2', '4', 2)
connect_nodes(NN_time, node_names, '2', '4', 3)

connect_nodes(NN_dist, node_names, '4', 't', 1)
connect_nodes(NN_time, node_names, '4', 't', 3)

connect_nodes(NN_dist, node_names, '2', 't', 5)
connect_nodes(NN_time, node_names, '2', 't', 1)

connect_nodes(NN_dist, node_names, 's', '3', 1)
connect_nodes(NN_time, node_names, 's', '3', 1)

connect_nodes(NN_dist, node_names, '3', '5', 2)
connect_nodes(NN_time, node_names, '3', '5', 3)

connect_nodes(NN_dist, node_names, '5', 't', 2)
connect_nodes(NN_time, node_names, '5', 't', 5)

In [5]:
NN = ((NN_dist!=0) | (NN_time!=0)!=0).astype(int)
pd.DataFrame((NN!=0).astype(int), columns=node_names, index=node_names)

,s,2,3,4,5,t
s,0,1,1,0,0,0
2,0,0,0,1,0,1
3,0,0,0,0,1,0
4,0,0,0,0,0,1
5,0,0,0,0,0,1
t,0,0,0,0,0,0


In [6]:
print("They are {} connections".format((NN!=0).astype(int).sum().sum()))

They are 7 connections


## Convert to solvable data

In [7]:
Aeq, arc_idxs = nn2na(NN_dist, node_names = node_names, show_results = False)

nan_names = get_col_names(NN, node_names, as_numpy=True, sep = "->")

print("Aeq:")
display(pd.DataFrame(Aeq, index=node_names, columns=nan_names))

t = get_costs(NN_time, arc_idxs)
A =  np.expand_dims(t, axis = 0)
display(pd.DataFrame(A, index=['t'], columns=nan_names))

# A, arc_idxs2 = nn2na(NN_time, node_names = node_names, show_results = False)

# print("A:")
# display(pd.DataFrame(A, index=node_names, columns=nan_names))

# print((arc_idxs == arc_idxs2))

Aeq:


,s->2,s->3,2->4,2->t,3->5,4->t,5->t
s,1,1,0,0,0,0,0
2,-1,0,1,1,0,0,0
3,0,-1,0,0,1,0,0
4,0,0,-1,0,0,1,0
5,0,0,0,0,-1,0,1
t,0,0,0,-1,0,-1,-1


,s->2,s->3,2->4,2->t,3->5,4->t,5->t
t,3.0,1.0,3.0,1.0,3.0,3.0,5.0


In [8]:
costs = get_costs(NN_dist, arc_idxs)

In [9]:
bounds = tuple([(0, None) for arcs in range(0, nan_names.shape[0])])

In [10]:
resume_df = pd.DataFrame(bounds, index=nan_names, columns=['Min bound', 'Max bound'])
resume_df['Costs'] = costs
resume_df['Max time'] = t
resume_df

,Min bound,Max bound,Costs,Max time
s->2,0,None,2.0,3.0
s->3,0,None,1.0,1.0
2->4,0,None,2.0,3.0
2->t,0,None,5.0,1.0
3->5,0,None,2.0,3.0
4->t,0,None,1.0,3.0
5->t,0,None,2.0,5.0


In [11]:
# Resumen
# print('## Optimizer inputs ## \n\n'
#       'Cost vector: %s \n\n'
#       'Columns: %s \n\n'
#       'A_eq Node-Arc matrix:\n%s \n\n'
#       'b_eq demand-supply vector: %s \n\n'
#       'Bounds of each X arc variable: %s \n' % (C, nan_names, Aeq, beq, bounds))

# Solve trough Simplex method for Tmax = 9

In [12]:
# Optimización
res_simplex = linprog(costs, A_eq=Aeq, A_ub = A, b_ub=T1, b_eq=beq, bounds=bounds, method='simplex')

selarcs = get_selected_arcs(res_simplex.x.round().astype(int), nan_names)

/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  


In [13]:
res_simplex.x

array([0., 1., 0., 0., 1., 0., 1.])

In [14]:
print('## Results ##')
print('The raw solution will be: %s' % res_simplex.x.round().astype(int))
print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f ' % res_simplex.fun)

## Results ##
The raw solution will be: [0 1 0 0 1 0 1]
The arcs that make the shortest path will be (from, to): ['s->3', '3->5', '5->t']
The minimum cost will be: 5.00 


In [15]:
resume_df['Solution T9'] = res_simplex.x.round().astype(int)

## Show resume for Tmax  = 9

In [16]:
resume_df['Costo total T9']=resume_df['Costs']*resume_df['Solution T9']
resume_df['Tiempo total T9']=resume_df['Max time']*resume_df['Solution T9']

In [17]:
print("La ruta detectada es ", selarcs)
print("El costo total es ", resume_df["Costo total T9"].sum())
print("El tiempo total es ", resume_df["Tiempo total T9"].sum())

La ruta detectada es  ['s->3', '3->5', '5->t']
El costo total es  5.0
El tiempo total es  9.0


In [18]:
np.sum(np.abs(res_simplex.x.round() - res_simplex.x))

0.0

In [19]:
resume_df.transpose()

,s->2,s->3,2->4,2->t,3->5,4->t,5->t
Min bound,0,0,0,0,0,0,0
Max bound,None,None,None,None,None,None,None
Costs,2,1,2,5,2,1,2
Max time,3,1,3,1,3,3,5
Solution T9,0,1,0,0,1,0,1
Costo total T9,0,1,0,0,2,0,2
Tiempo total T9,0,1,0,0,3,0,5


In [20]:
res_simplex

     con: array([0., 0., 0., 0., 0., 0.])
     fun: 5.0
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([0.])
  status: 0
 success: True
       x: array([0., 1., 0., 0., 1., 0., 1.])

# Solve trough Simplex method for Tmax = 8

In [26]:
# Optimización
res_simplex = linprog(costs, A_eq=Aeq, A_ub = A, b_ub=T2, b_eq=beq, bounds=bounds, method='simplex')

selarcs = get_selected_arcs(res_simplex.x.round().astype(int), nan_names)

/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  


In [27]:
res_simplex.x

array([0.2, 0.8, 0. , 0.2, 0.8, 0. , 0.8])

In [28]:
print('## Results ##')
print('The raw solution will be: %s' % res_simplex.x.round().astype(int))
print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f ' % res_simplex.fun)

## Results ##
The raw solution will be: [0 1 0 0 1 0 1]
The arcs that make the shortest path will be (from, to): ['s->3', '3->5', '5->t']
The minimum cost will be: 5.40 


In [30]:
resume_df['Solution T8'] = res_simplex.x.round().astype(int)

## Show resume for Tmax  = 8

In [31]:
resume_df['Costo total T8']=resume_df['Costs']*resume_df['Solution T8']
resume_df['Tiempo total T8']=resume_df['Max time']*resume_df['Solution T8']

In [32]:
print("La ruta detectada es ", selarcs)
print("El costo total es ", resume_df["Costo total T8"].sum())
print("El tiempo total es ", resume_df["Tiempo total T8"].sum())

La ruta detectada es  ['s->3', '3->5', '5->t']
El costo total es  5.0
El tiempo total es  9.0


In [33]:
np.sum(np.abs(res_simplex.x.round() - res_simplex.x))

1.0000000000000004

In [34]:
resume_df.transpose()

,s->2,s->3,2->4,2->t,3->5,4->t,5->t
Min bound,0,0,0,0,0,0,0
Max bound,None,None,None,None,None,None,None
Costs,2,1,2,5,2,1,2
Max time,3,1,3,1,3,3,5
Solution T9,0,1,0,0,1,0,1
Costo total T9,0,1,0,0,2,0,2
Tiempo total T9,0,1,0,0,3,0,5
Solution T8,0,1,0,0,1,0,1
Costo total T8,0,1,0,0,2,0,2
Tiempo total T8,0,1,0,0,3,0,5
